In [1]:
import pandas as pd

In [3]:
df_gs = pd.read_csv('titanic_data/gender_submission.csv')
df_test = pd.read_csv('titanic_data/test.csv')
df_train = pd.read_csv('titanic_data/train.csv')

In [3]:
df_train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
print(len(df_train))

891


In [5]:
### 欠損値の確認
missing_counts = df_train.isnull().sum()
missing_counts

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Cabin（客室番号）の欠損が多い。
しかし家族なら同じ客室番号ではないだろうか。

In [6]:
pd.pivot_table(df_train, index='Survived', columns='Pclass', values='Age', aggfunc=len)

Pclass,1,2,3
Survived,,,
0,80,97,372
1,136,87,119


In [7]:
pd.pivot_table(df_train, index='Survived', columns='Embarked', values='Age', aggfunc=len)

Embarked,C,Q,S
Survived,,,
0,75,47,427
1,93,30,217


In [8]:
pd.pivot_table(df_train, index='Pclass', columns='Embarked', values='Age', aggfunc=len)

Embarked,C,Q,S
Pclass,,,
1,85,2,127
2,17,3,164
3,66,72,353


Nameの観察

In [16]:
### Mr などの種類の把握 ###
import pandas as pd
import re

# 最初のカンマまでの文字列を抽出する関数
def extract_before_comma(name):
    comma_index = name.find(",")
    return name[:comma_index].strip() if comma_index != -1 else None

# 新しいカラム 'FamilyName' に抽出結果を入れる
df_train['FamilyName'] = df_train['Name'].apply(extract_before_comma)

# 出現回数をカウント
FN_counts = df_train['FamilyName'].value_counts()

print(FN_counts.head(30))
print(len(FN_counts))

FamilyName
Andersson     9
Sage          7
Panula        6
Skoog         6
Carter        6
Goodwin       6
Johnson       6
Rice          5
Lefebre       4
Asplund       4
Harris        4
Fortune       4
Baclini       4
Brown         4
Hart          4
Kelly         4
Harper        4
Smith         4
Gustafsson    4
Palsson       4
Williams      4
Ford          4
Johansson     3
Davies        3
Jussila       3
Flynn         3
Hickman       3
Thayer        3
West          3
Laroche       3
Name: count, dtype: int64
667


In [ ]:
df_train[df_train['FamilyName']=='Andersson']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilyName
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S,Mr,Andersson
68,69,1,3,"Andersson, Miss. Erna Alexandra",female,17.0,4,2,3101281,7.9250,NaN,S,Miss,Andersson
119,120,0,3,"Andersson, Miss. Ellis Anna Maria",female,2.0,4,2,347082,31.2750,NaN,S,Miss,Andersson
146,147,1,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,27.0,0,0,350043,7.7958,NaN,S,Mr,Andersson
541,542,0,3,"Andersson, Miss. Ingeborg Constanzia",female,9.0,4,2,347082,31.2750,NaN,S,Miss,Andersson
542,543,0,3,"Andersson, Miss. Sigrid Elisabeth",female,11.0,4,2,347082,31.2750,NaN,S,Miss,Andersson
610,611,0,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",female,39.0,1,5,347082,31.2750,NaN,S,Mrs,Andersson
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S,Miss,Andersson
850,851,0,3,"Andersson, Master. Sigvard Harald Elias",male,4.0,4,2,347082,31.2750,NaN,S,Master,Andersson


In [ ]:
### もしかしてチケットで同行かわかる？
Ticket_counts = df_train['Ticket'].value_counts()
Ticket_counts.head(30)

Ticket
347082           7
CA. 2343         7
1601             7
3101295          6
CA 2144          6
347088           6
S.O.C. 14879     5
382652           5
LINE             4
PC 17757         4
17421            4
349909           4
113760           4
4133             4
113781           4
W./C. 6608       4
2666             4
19950            4
347077           4
C.A. 31921       3
PC 17572         3
C.A. 34651       3
363291           3
F.C.C. 13529     3
345773           3
248727           3
24160            3
29106            3
SC/Paris 2123    3
35273            3
Name: count, dtype: int64

もしかしたらチケットと家族はほぼ同じ情報か？
検証の価値あり。